In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from utils import import_data_to_matrix_split, import_data_to_matrix, extract_submission
from utils import get_rmse_score

# Global Bias Method
- Idea: Each user and each item has a specific bias towards ratings

## Data Preprocessings
- Extract data to row-column format
- Impute missing data with 0
- Rating matrix A
- Observation matrix Ω
  
## Global Bias
- Global rating mean:
$$μ = \frac{\sum_{u=1}^n \sum_{i=1}^m ω_{ui}a_{ui}}{\sum_{u=1}^n \sum_{i=1}^m ω_{ui}}$$
- User $u$ rating mean:
$$μ_{(u,.)} = \frac{\sum_{i=1}^m ω_{ui}a_{ui}}{\sum_{i=1}^m ω_{ui}}$$
- Item $i$ rating mean:
$$μ_{(.,i)} = \frac{\sum_{u=1}^n ω_{ui}a_{ui}}{\sum_{u=1}^n ω_{ui}}$$
- User $u$ rating bias: (Initial values for ALS)
$$b_{(u,.)} = μ_{(u,.)} - \frac{\sum_{v=1}^n μ_{(v,.)}}{n}$$
- Item $i$ rating bias: (Initial values for ALS)
$$b_{(.,i)} = μ_{(.,i)} - \frac{\sum_{j=1}^m μ_{(.,j)}}{m}$$

## ALS
- Optimize global bias
  
- Objective function:
$$l(B_u, B_i) = \frac{1}{2}||Π_{Ω}(A - μ^{n⨉m} - B_u·1^{1⨉m} - (B_i·1^{1⨉n})^T)||_{F}^{2} + \frac{λ}{2}(||B_u||_{2}^{2} + ||B_i||_{2}^{2})$$

- $B_u$ is the column vector for all $b_{(u,.)}$ ($b_{(1,.)},...,b_{(n,.)}$)
- $B_i$ is the column vector for all $b_{(.,i)}$ ($b_{(.,1)},...,b_{(.,m)}$)

- Focus on the contribution to the error of a single $b_{(u,.)}$:
- Where does $b_{(u,.)}$ appear in error?
$$l_{B_i}(b_{(u,.)}) = \frac{1}{2}\sum_{i=1}^m ω_{ui}(a_{ui} - μ - b_{(u,.)} - b_{(.,i)})^2 + \frac{λ}{2}b_{(u,.)}^2$$

- Partial derivative with respect to $b_{(u,.)}$
$$\frac{∂l}{∂b_{(u,.)}} = \frac{1}{2}\sum_{i=1}^m 2ω_{ui}(a_{ui} - μ - b_{(u,.)} - b_{(.,i)})(-1) + \frac{λ}{2}2b_{(u,.)}$$

- Set $\frac{∂l}{∂b_{(u,.)}} = 0$
$$0 = -\sum_{i=1}^m ω_{ui}(a_{ui} - μ - b_{(u,.)} - b_{(.,i)}) + λb_{(u,.)}$$
$$λb_{(u,.)} = \sum_{i=1}^m ω_{ui}(a_{ui} - μ - b_{(.,i)}) - \sum_{i=1}^m ω_{ui}b_{(u,.)}$$
$$λb_{(u,.)} + \sum_{i=1}^m ω_{ui}b_{(u,.)} = \sum_{i=1}^m ω_{ui}(a_{ui} - μ - b_{(.,i)})$$
$$b_{(u,.)}(λ + \sum_{i=1}^m ω_{ui}) = \sum_{i=1}^m ω_{ui}(a_{ui} - μ - b_{(.,i)})$$

- Thus,
$$b_{(u,.)}^* = (λ + \sum_{i=1}^m ω_{ui})^{-1}\sum_{i=1}^m ω_{ui}(a_{ui} - μ - b_{(.,i)})$$
- Note that $(λ + \sum_{i=1}^m ω_{ui}) > 0$ because $ω_{ui}∈\{0,1\}$ and $λ > 0$

- Equations for all $b_{(u,.)}$, $1<=u<=n$
$$(λ + \sum_{i=1}^m ω_{1i})b_{(1,.)} = \sum_{i=1}^m ω_{1i}(a_{1i} - μ - b_{(.,i)})$$
$$⋮$$
$$(λ + \sum_{i=1}^m ω_{ni})b_{(n,.)} = \sum_{i=1}^m ω_{ni}(a_{ni} - μ - b_{(.,i)})$$

- Thus,
$$\begin{bmatrix} (λ + \sum_{i=1}^m ω_{1i}) \\ ⋮ \\ (λ + \sum_{i=1}^m ω_{ni}) \end{bmatrix}\begin{bmatrix} b_{(1,.)}^* \\ ⋮ \\ b_{(n,.)}^* \end{bmatrix} = \begin{bmatrix} \sum_{i=1}^m ω_{1i}(a_{1i} - μ - b_{(.,i)}) \\ ⋮ \\ \sum_{i=1}^m ω_{ni}(a_{ni} - μ - b_{(.,i)}) \end{bmatrix}$$
- Which is equivalent to,
$$(λ^{n⨉n} + diag(Ω·1^{m⨉1}))\begin{bmatrix} b_{(1,.)}^* \\ ⋮ \\ b_{(n,.)}^* \end{bmatrix} = (Π_Ω(A - μ^{n⨉m} - 1^{n⨉1}·B_i^T))·1^{m⨉1}$$

- Similarly for all $b_{(.,i)}$,
$$(λ^{m⨉m} + diag(Ω^T·1^{n⨉1}))\begin{bmatrix} b_{.,1)}^* \\ ⋮ \\ b_{(.,m)}^* \end{bmatrix} = (Π_{Ω^T}(A^T - μ^{m⨉n} - 1^{m⨉1}·B_u^T))·1^{n⨉1}$$

- Global estimation for rating of item $i$ by user $u$:
$$b_{ui} = μ + b_{(u,.)} + b_{(.,i)}$$

In [ ]:
class GBias():
    
    def __init__(self, A, lambda1=0.001, epochs=5):
        self.A = A
        self.W = (self.A > 0).astype(int)
        self.num_users, self.num_items = self.A.shape
        self.lambda1 = lambda1
        self.epochs = epochs
        
        # Initialize the biases
        self.global_mean = np.sum(self.W * self.A)/np.sum(self.W)
        Mu = np.array([np.sum(Wu * Au)/np.sum(Wu) for Au, Wu in zip(self.A, self.W)])
        Mi = np.array([np.sum(Wi * Ai)/np.sum(Wi) for Ai, Wi in zip(self.A.T, self.W.T)])

        self.Bu = Mu - np.mean(Mu)
        self.Bi = Mi - np.mean(Mi)

        self.Bu = np.reshape(self.Bu, (self.Bu.shape[0],1))
        self.Bi = np.reshape(self.Bi, (self.Bi.shape[0],1))
    
    def _loss(self):
        return ((1/2) * np.sum((self.W * (self.A - self.global_mean - np.dot(self.Bu, np.ones((1, self.num_items))) - np.dot(self.Bi, np.ones((1, self.num_users))).T) ** 2))
                + (self.lambda1/2) * (np.sum(self.Bu ** 2) + np.sum(self.Bi ** 2)))
    
    def train(self, test_matrix=None):
        error_progress = {
            "train_rmse": [],
            "test_rmse": [],
        }
        for epoch in tqdm(range(self.epochs)):
            self._als()
            rec_A = self.reconstruct_matrix()
            train_rmse = get_rmse_score(rec_A, self.A)
            error_progress["train_rmse"].append(train_rmse)
            if test_matrix is not None:
                test_rmse = get_rmse_score(rec_A, test_matrix)
                error_progress["test_rmse"].append(test_rmse)
            # print(error_progress)
        return error_progress
    
    def _als(self):
        self.Bi = np.linalg.solve(self.lambda1 + np.diag(np.dot(self.W.T, np.ones((self.num_users,1))).T[0]),
                                    np.dot(
                                        self.W.T * (self.A.T - self.global_mean - np.dot(np.ones((self.num_items, 1)), self.Bu.T)),
                                        np.ones((self.num_users, 1))
                                    )
                                )
        # print("Loss l(Bu,Bi) after solving for Bi:", self._loss())

        self.Bu = np.linalg.solve(self.lambda1 + np.diag(np.dot(self.W, np.ones((self.num_items,1))).T[0]),
                                np.dot(
                                    self.W * (self.A - self.global_mean - np.dot(np.ones((self.num_users, 1)), self.Bi.T)),
                                    np.ones((self.num_items, 1))
                                )
                            )
        # print("Loss l(Bu,Bi) after solving for Bu:", self._loss())
    
    def reconstruct_matrix(self):
        """
        Compute the full matrix using biases
        """
        biases = self.global_mean + np.array([self.Bu.T[0]]*self.num_items).T + np.array([self.Bi.T[0]]*self.num_users)
        return biases

In [ ]:
# A, test_matrix = import_data_to_matrix_split()
# model = GBias(A, lambda1=0.001, epochs=5)
# model.train(test_matrix=test_matrix)

In [ ]:
A = import_data_to_matrix()
model = GBias(A, lambda1=0.001, epochs=5)
model.train()
rec_A = model.reconstruct_matrix()

In [ ]:
rec_A[rec_A>5] = 5
rec_A[rec_A<1] = 1

In [ ]:
extract_submission(rec_A, file="global")